# Data Analysis 2

This notebook is ***NEW CONTINUING*** from previous notebooks in the [notebooks](https://github.com/Data-Science-for-Linguists-2023/For-Reddit-Grammaticality-Analysis/tree/main/notebooks) folder. Therefore, before reading through this notebook, I recommend going through [Data Collection](https://github.com/Data-Science-for-Linguists-2023/For-Reddit-Grammaticality-Analysis/blob/main/notebooks/dataCollection.ipynb), [Data Organization](https://github.com/Data-Science-for-Linguists-2023/For-Reddit-Grammaticality-Analysis/blob/main/notebooks/dataOrganization.ipynb), and then [Data Analysis 1](https://github.com/Data-Science-for-Linguists-2023/For-Reddit-Grammaticality-Analysis/blob/main/notebooks/dataAnalysis1.ipynb), in that order.

## Setting Up

In [1]:
# Start with importing our libraries
import language_tool_python as ltp  # Using this as a grammaticality parser
import pandas as pd 
import numpy as np
import nltk
import re

In [2]:
# This sets up our parsing tool
tool = ltp.LanguageTool('en-US')

In [3]:
# Using pandas to read csv files in 

legalAdvice = pd.read_csv("../final-data/finalLegalData.csv", index_col = 0)
adulting = pd.read_csv('../final-data/finalAdData.csv', index_col = 0)
medicine = pd.read_csv('../final-data/finalMedData.csv', index_col = 0)
highschool = pd.read_csv('../final-data/finalHsData.csv', index_col = 0)
broadway = pd.read_csv('../final-data/finalBwayData.csv', index_col = 0)
pittsburgh = pd.read_csv('../final-data/finalPghData.csv', index_col = 0)
rant = pd.read_csv('../final-data/finalRantData.csv', index_col = 0)
ccq = pd.read_csv('../final-data/finalCcqData.csv', index_col = 0)
anime = pd.read_csv('../final-data/finalAnimeData.csv', index_col = 0)
eli5 = pd.read_csv('../final-data/finalElifData.csv', index_col = 0)
college = pd.read_csv('../final-data/finalCollegeData.csv', index_col = 0)
sports = pd.read_csv('../final-data/finalSportsData.csv', index_col = 0)
crypto = pd.read_csv('../final-data/finalCryptoData.csv', index_col = 0)
lawyertalk = pd.read_csv('../final-data/finalLawyerData.csv', index_col = 0)
gaming = pd.read_csv('../final-data/finalGamingData.csv', index_col = 0)

In [4]:
crypto

,Title,Id,Text,Author,Number of Comments,Number of Upvotes,Ratio of Upvotes
0,Moon Week 37,11t058d,Hello everyone and welcome to Moon Week for ro...,MoonWeek,668,102,0.88
1,"Daily General Discussion - March 19, 2023 (GMT+0)",11v58f9,**Welcome to the Daily General Discussion thre...,CryptoDaily-,5390,82,0.94
2,I went to the supermarket here in Venezuela an...,11vnevt,"Hi guys, as you probably know I'm Venezuelan l...",WorkingLime,630,979,0.85
3,Having An Emergency Fund Is One Of The Most Im...,11vqd67,I rarely hear people recommend having an emerg...,kirtash93,172,87,0.80
4,I just realized i lost about 900$,11vw1ps,"I was kinda active at one time in here, moved ...",KermitTheFrogo01,88,30,0.86
...,...,...,...,...,...,...,...
418,Don't Let Hackers Drain Your Crypto Wallets: S...,11t0wcb,I decided to create this post because of the e...,kirtash93,36,15,0.70
419,Storing Your Seed Phrase (give me a better ide...,11t0j05,"Obviously, there are a lot of opinions on how ...",Nonchalant_Calypso,216,25,0.75
420,Noob question about cold storage,11t0bda,I've looked into it and I still have some ques...,Randompersonxxxx,26,5,0.78
421,Moon Week 37,11t058d,Hello everyone and welcome to Moon Week for ro...,MoonWeek,697,113,0.89


In [5]:
allSportsVals = list(sports['Text'].values)
allLegalVals = list(legalAdvice['Text'].values)
allAdultVals = list(adulting['Text'].values)
allMedVals = list(medicine['Text'].values)
allHsVals = list(highschool['Text'].values)
allBwayVals = list(broadway['Text'].values)
allPghVals = list(pittsburgh['Text'].values)
allRantVals = list(rant['Text'].values)
allCcqVals = list(ccq['Text'].values)
allAnimeVals = list(anime['Text'].values)
allEli5Vals = list(eli5['Text'].values)
allCollegeVals = list(college['Text'].values)
allCryptoVals = list(crypto['Text'].values)
allLawyerVals = list(lawyertalk['Text'].values)
allGamingVals = list(gaming['Text'].values)

sportsErrors, legalErrors, adultErrors, medErrors, hsErrors, bwayErrors, pghErrors, rantErrors, ccqErrors, animeErrors, eli5Errors, collegeErrors, cryptoErrors, lawyerErrors, gamingErrors = [], [], [], [], [], [], [], [], [], [], [], [], [], [], []


In [6]:
[sportsErrors.append(tool.check(x)) for x in allSportsVals]
[legalErrors.append(tool.check(x)) for x in allLegalVals]
[adultErrors.append(tool.check(x)) for x in allAdultVals]
[medErrors.append(tool.check(x)) for x in allMedVals]
[hsErrors.append(tool.check(x)) for x in allHsVals]
[bwayErrors.append(tool.check(x)) for x in allBwayVals]
[pghErrors.append(tool.check(x)) for x in allPghVals]
[rantErrors.append(tool.check(x)) for x in allRantVals]
[ccqErrors.append(tool.check(x)) for x in allCcqVals]
[animeErrors.append(tool.check(x)) for x in allAnimeVals]
[eli5Errors.append(tool.check(x)) for x in allEli5Vals]
[collegeErrors.append(tool.check(x)) for x in allCollegeVals]
[cryptoErrors.append(tool.check(x)) for x in allCryptoVals]
[lawyerErrors.append(tool.check(x)) for x in allLawyerVals]
[gamingErrors.append(tool.check(x)) for x in allGamingVals]
print("Done processing")

Done processing


for later reference, let's show the errors, and then remove morfologik rules because most of them are just typos, and that doesn't really tell us much. let's all remove similar errors, such as the pokemon rule, etc and keep going through typos that appear in top 10 until we have a set of top 10 errors that make sense for our focus. After that, make some graphs. Also maybe want to try and see if the tool has any pitfalls. can we get an accuracy score? is that too ambitious? could we get the # of sentences and calculate how many sentences have errors over total number of errors

## Where Do We Go With Our Analysis?

After our analysis last time, we discovered the most frequently occuring grammatical errors in each of the subreddits. Here it is again for reference:

**COMMA_COMPOUND_SENTENCE** - Legal Advice, Adulting, College

**MORFOLOGIK_RULE_EN_US** - Medicine, Highschool, Broadway, Pittsburgh, Rant, Ccq, Anime, Eli5, Sports, Cryptocurrency, Lawyer Talk, Gaming

Let's take a closer look at what each of these errors looks like.

In [7]:
hsErrors[6]

[Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'message': 'Possible spelling mistake found.', 'replacements': ['RN', 'Rn', 'in', 'on', 'an', 'run', 'ran', 'RNA', 'urn', 'RCN', 'RJ', 'RR', 'RPN', 'Ru', 'VN', 'RNG', 'Rx', 'Zn', 'rt', 'Ron', 'AN', 'ARN', 'BRN', 'CN', 'CRN', 'DN', 'EN', 'FN', 'GN', 'GRN', 'IN', 'IRN', 'In', 'KN', 'LN', 'Ln', 'MN', 'Mn', 'N', 'ON', 'ORN', 'PRN', 'R', 'RC', 'RD', 'RE', 'RF', 'RFN', 'RI', 'RM', 'RMN', 'RN6', 'RNB', 'RNE', 'RNF', 'RNN', 'RNO', 'RNR', 'RNS', 'RNT', 'RNU', 'RO', 'RP', 'RRN', 'RS', 'RT', 'RTN', 'RU', 'RUN', 'RV', 'Ra', 'Rb', 'Rd', 'Re', 'Rf', 'Rh', 'Ry', 'SN', 'Sn', 'TN', 'UN', 'URN', 'en', 'kn', 'n', 'r', 'rd', 're', 'rm', 'rs', 'tn', 'NRN', 'PN', 'RL', 'RNC', 'RNZ', 'RQ', 'RX', 'kN'], 'offsetInContext': 3, 'context': 'So rn I’m in eighth grade, and next year I’m ...', 'offset': 3, 'errorLength': 2, 'category': 'TYPOS', 'ruleIssueType': 'misspelling', 'sentence': "So rn I'm in eighth grade, and next year I'm going to a pretty prestigious/rigorous sc

As seen above, we have two MORFOLOGIK_RULE_EN_US rules broken. Specifically, with ruleIssueType of 'misspelling'. Let's explore just how prominent 'misspelling' ruleIssueTypes are.

In [16]:
counter = 0
counterAll = 0
for x in sportsErrors:
    if x:
        counterAll += 1
        for y in x:
            if y.ruleIssueType == 'misspelling':
                counter += 1
                break

for x in legalErrors:
    if x:
        counterAll += 1
        for y in x:
            if y.ruleIssueType == 'misspelling':
                counter += 1
                break
                
for x in adultErrors:
    if x:
        counterAll += 1
        for y in x:
            if y.ruleIssueType == 'misspelling':
                counter += 1
                break
                
for x in medErrors:
    if x:
        counterAll += 1
        for y in x:
            if y.ruleIssueType == 'misspelling':
                counter += 1
                break
                
for x in hsErrors:
    if x:
        counterAll += 1
        for y in x:
            if y.ruleIssueType == 'misspelling':
                counter += 1
                break
                
for x in bwayErrors:
    if x:
        counterAll += 1
        for y in x:
            if y.ruleIssueType == 'misspelling':
                counter += 1
                break
            
for x in pghErrors:
    if x:
        counterAll += 1
        for y in x:
            if y.ruleIssueType == 'misspelling':
                counter += 1
                break
            
for x in rantErrors:
    if x:
        counterAll += 1
        for y in x:
            if y.ruleIssueType == 'misspelling':
                counter += 1
                break
            
for x in ccqErrors:
    if x:
        counterAll += 1
        for y in x:
            if y.ruleIssueType == 'misspelling':
                counter += 1
                break
            
for x in animeErrors:
    if x:
        counterAll += 1
        for y in x:
            if y.ruleIssueType == 'misspelling':
                counter += 1
                break
            
for x in eli5Errors:
    if x:
        counterAll += 1
        for y in x:
            if y.ruleIssueType == 'misspelling':
                counter += 1
                break
            
for x in collegeErrors:
    if x:
        counterAll += 1
        for y in x:
            if y.ruleIssueType == 'misspelling':
                counter += 1
                break
            
for x in cryptoErrors:
    if x:
        counterAll += 1
        for y in x:
            if y.ruleIssueType == 'misspelling':
                counter += 1
                break
            
for x in lawyerErrors:
    if x:
        counterAll += 1
        for y in x:
            if y.ruleIssueType == 'misspelling':
                counter += 1
                break
            
for x in gamingErrors:
    if x:
        counterAll += 1
        for y in x:
            if y.ruleIssueType == 'misspelling':
                counter += 1
                break
            
print(counter)
print(counterAll)

13509
18352


Wow. That means in all of the 18352 errors, 13509 of them are misspelling errors. That is 73.61% of all of the errors. Let's explore some of these misspellinng errors.